## Least-Squares Monte Carlo

### Import libraries

In [1]:
import numpy as np
import pandas as pd
from numpy.polynomial import laguerre

### 1. Monte-Carlo European Basket Call Option

#### Input Parameters

In [2]:
r = 0.05
delta = 0.02
sigma = 0.3
rho = 0.2
N = 50000
T = 1
S0 = K = 100
rho = np.array([[1,0.2,0.2],[0.2,1,0.2],[0.2,0.2,1]])

#### Implement the Monte Carlo Method for European Basket Option

In [3]:
# Generate a (3, N) weiner process
dW = np.random.randn(3, N)*np.sqrt(T)

# Compute the Cholesky decomposition of the correlation matrix rho
L = np.linalg.cholesky(rho)

# Calculating stock price for 3 assets at final maturity for N simulations
St = S0*np.exp((r-delta-0.5*sigma**2)*T + sigma*np.dot(L,dW))

#### Compute Weights

$$\lambda_t = \frac{S_{t}}{\sum_{i=1}^{3}S_{i,t}}$$

In [4]:
# Calculate capitalization weights and prices for each of the 3 assets
total = np.sum(St, axis=0)
lambda_ = St/total

#### Compute European Basket Option Price
$$V_0 = \mathbb{E}^{Q}[\max\left(\sum_{i=1}^{3}\lambda_iS_{i,T}-K,0\right)]e^{-rT}$$

In [5]:
# Calculate basket option price
price_ = lambda_*St
Basket = np.sum(price_, axis=0)
payoff = np.maximum(Basket-K, 0)*np.exp(-r*T)
Basket_Option_Price = np.mean(payoff)
print(f"European Basket Option Price for 3 Assets: {Basket_Option_Price:.4f}")

European Basket Option Price for 3 Assets: 12.7426


### 2. Least Squares Monte Carlo (Multi-Index & Ad-hoc) for Bermudan Basket Call Option

#### Input Parameters

In [6]:
r1 = 0.05
delta1 = [0.02, 0.02, 0.02]
sigma1 = [0.3, 0.3, 0.3]
rho1 = [[1.0, 0.2, 0.2], [0.2, 1.0, 0.2], [0.2, 0.2, 1.0]]
S01 = 100
K1 = 100.0
T1 = 1.0
N1 = 50000
d1 = 10
n1 = 12
dt1 = T1/n1

#### Simulate Stock Prices

In [7]:
# Generate standard normal random variables
Z = np.random.normal(size=(n1, 3, N1))

# Compute the Cholesky decomposition of the correlation matrix
L1 = np.linalg.cholesky(rho1)

# Initialise vector for stock prices
S1 = np.zeros((n1+1, 3, N1))
S1[0] = S01 # Set stock price at time 0 as S0

# Simulate the asset prices using the geometric Brownian motion equation
for i in range(1, n1+1):
    dW1 = np.dot(L1, Z[i-1])
    for j in range(3):
        S1[i,j] = S1[i-1,j]*np.exp((r1-delta1[j]-0.5*sigma1[j]**2)*dt1 + sigma1[j]*np.sqrt(dt1)*dW1[j])

#### Compute Weighted Price of each asset

In [8]:
# Compute the sum of S_squared along axis 1
sum_S = np.sum(S1, axis=1)

# Square all values in S
S_squared = S1 ** 2

# Divide S_squared by the sum_S_squared array using broadcasting
S_T = (S_squared / sum_S[:, np.newaxis, :])

#### Function: Bermudan Call Option for Multi Index using LSMC

In [9]:
def bermudan_call_LSMC_multi_index(K, S_t, S, T, r, n):
    ## LSMC model parameters
    dt = T / n           
    df = np.exp(-r*dt)
    d = 10               
    
    # Sum along axis 1 to get an array of shape (13, 50000)
    S_T = np.sum(S_t, axis=1)
    
    ## Initialise arrays
    payoff = np.maximum(S_T-K, 0)    
    V = np.zeros_like(S_T)            
    V[-1, :] = payoff[-1, :]         
    
    ## Iterate over time steps backwards
    for t in range(n, 0, -1):        
        V[t-1,:] = V[t,:] * df       
        ind = payoff[t - 1,:] > 0    
        
        ## Create the polynomial basis matrix using Laguerre polynomials
        psi = np.zeros((np.sum(ind),286))
        l = 0
        for i in range(d+1):
            for j in range(d+1-i):
                for k in range(d+1-i-j):
                    p1 = laguerre.Laguerre.basis(i).coef
                    p2 = laguerre.Laguerre.basis(j).coef
                    p3 = laguerre.Laguerre.basis(k).coef
                    psi[:,l] = laguerre.lagval(S[t-1,0,ind], p1)*laguerre.lagval(S[t-1,1,ind], p2)*laguerre.lagval(S[t-1,2,ind], p3)
                    l = l+1
        q_psi, r_psi = np.linalg.qr(psi)                 
        b = q_psi.T @ V[t, ind] * df  
        CV = q_psi @ b                
        V[t - 1, ind] = np.maximum(CV, payoff[t-1, ind]) 

    return V 

#### Function: Bermudan Call Option using Ad Hoc LSMC

In [10]:
def bermudan_call_LSMC_ad_hoc(K, S, T, r, n):
    ## LSMC model parameters
    dt = T / n           
    df = np.exp(-r*dt)   
    d = 10               
    
    ## Initialise arrays
    payoff = np.maximum(S-K, 0)    
    V = np.zeros_like(S)             
    V[-1, :] = payoff[-1, :]         
    
    ## Iterate over time steps backwards
    for t in range(n, 0, -1):        
        V[t-1,:] = V[t,:] * df       
        ind = payoff[t - 1,:] > 0    
        
        ## Create the polynomial basis matrix using Laguerre polynomials
        psi = np.zeros((np.sum(ind), d+1))
        for i in range(d+1):
            p = laguerre.Laguerre.basis(i).coef          
            psi[:,i] = laguerre.lagval(S[t-1, ind], p)   
        q_psi, r_psi = np.linalg.qr(psi)                 
        b = q_psi.T @ V[t, ind] * df  
        CV = q_psi @ b                
  
        # Set maximum of continuation value (CV) or payoff as the value of option at each node for current time step
        V[t - 1, ind] = np.maximum(CV, payoff[t-1, ind]) 

    return V 

#### Compute Multi-index LSMC Price

In [11]:
V_mi = bermudan_call_LSMC_multi_index(K1, S_T, S1, T1, r1, n1)
print(f"Bermudan Basket Option Price for Multi Index using LSMC : {V_mi[0,:].mean():.4f}")

Bermudan Basket Option Price for Multi Index using LSMC : 12.9298


#### Weighted Prices for Ad-hoc

In [12]:
numerator = np.sum(np.power(S1, 2), axis=1)
denominator = np.sum(S1, axis=1)
S_adhoc = numerator / denominator

#### Compute Ad-hoc LSMC Price

In [13]:
V_adhoc = bermudan_call_LSMC_ad_hoc(K1, S_adhoc, T1, r1, n1)
print(f"Bermudan Basket Option Price using ad-hoc LSMC : {V_adhoc[0,:].mean():.4f}")

Bermudan Basket Option Price using ad-hoc LSMC : 12.8187


#### Results

In [14]:
data = {'European Call MC': [f"{Basket_Option_Price:.4f}"],
        'LSMC Multi-Index Bermudan': [f"{V_mi[0,:].mean():.4f}"],
        'Ad-hoc LSMC Bermudan': [f"{V_adhoc[0,:].mean():.4f}"]}

Prices_Table = pd.DataFrame(data)

# Set the index to an empty list
Prices_Table.index = ['']

# Print the DataFrame
Prices_Table

,European Call MC,LSMC Multi-Index Bermudan,Ad-hoc LSMC Bermudan
,12.7426,12.9298,12.8187


### Conclusion

The Bermudan LSMC Price is higher than both european call option and ad-hoc LSMC Bermudan Option. The Bermudan option can be exercised early at any of the 12 time steps hence adding early exercise premium to it's price. The ad-hoc LSMC gives a rough estimate of Bermudan Option but computation time is much faster.